In [2]:
import pandas as pd
from os.path import exists
import matplotlib.pyplot as plt

In [74]:
class GoldAnalysis():
    """
        analysis of the price of gold in relation to the increase in the monetary base.
    """
    def __init__(self):
        try:
            self.df = pd.read_csv(
                filepath_or_buffer=f'../data/df.csv',
                sep=',',
            )

        except FileNotFoundError:
            print('O dataframe não existi, ultilize a função creating_dataframe().')

    def creating_monetary_base_dataframe(self):
        df = pd.read_csv(
            filepath_or_buffer='../data/BOGMBASE.csv',
            parse_dates=['DATE'],
            decimal='.',
        )

        df['year'] = df['DATE'].dt.year
        grouped = df.groupby('year')['BOGMBASE'].mean()
        result = grouped.reset_index()
        result = result.rename(
            columns={
                'BOGMBASE': 'average_monetary_base'
            }
        )
        result.drop(
            result.index[[-1, -2, -3, -4, -5, -6]],
            inplace=True,
        )
        self.df_monetary_base = result

    def creating_historic_gold_dataframe(self):
        df = pd.read_csv(
            filepath_or_buffer=r'../data/historic_gold_prices_1833_pres.csv',
            sep=',',
            decimal='.',
        )

        df.drop(
            df.index[:126],
            inplace=True,
        )
        result = df.rename(
            columns={
                'Year': 'year',
                'Average_Price': 'average_gold_price',
            }
        )

        self.df_gold = result

    def creating_dataframe(self):
        if not exists(r'../data/df.csv'):
            self.creating_monetary_base_dataframe()
            self.creating_historic_gold_dataframe()

            df = self.df_gold.join(
                self.df_monetary_base.set_index('year'),
                on='year',
            )

            df.to_csv(
                path_or_buf=r'../data/df.csv',
                index=False,
            )

        else:
            print("O arquivo já existi.")

    def creating_m2_dataframe(self):
        """
            Antes:
                DATE,M2SL
                1959-01-01,286.6
                1959-02-01,287.7
                1959-03-01,289.2
                1959-04-01,290.1
                1959-05-01,292.2
                1959-06-01,294.1
                1959-07-01,295.2
                1959-08-01,296.4
                1959-09-01,296.7
                1959-10-01,296.5
                1959-11-01,297.1
                1959-12-01,297.8

            Depois:
                year, average_m2
                1950, 295.2
        """
        df = pd.read_csv(
            filepath_or_buffer=r'../data/M2SL.csv',
            sep=',',
        )
        # Converta a coluna DATE em tipo de data
        df["DATE"] = pd.to_datetime(df["DATE"])

        # Adicione uma coluna para o ano
        df["year"] = df["DATE"].dt.year

        # Agrupe os dados por ano e calcule a média da coluna M2SL
        grouped = df.groupby("year")["M2SL"].mean()

        # Converte o resultado em um dataframe
        result = grouped.reset_index()

        # Renomeie a coluna resultante
        result = result.rename(columns={"M2SL": "average_m2"})

        # Salve o resultado em uma variável
        self.df_m2 = result

    def increase_in_new_dollar_creation_per_year_in_percentage(self):
        """
            A parti da média do m2 de 1959, quais foram os aumentos 
            tendo como base o m2 do ano anterio.

            Exemplo:
                DATE,M2SL
                1959 ,286.6 - 0%  _
                1960 ,312.4 - 30% _|
                1961 ,335.5 - 10% _|
        """
        self.creating_m2_dataframe()

        for index, value in enumerate(self.df_m2['average_m2']):
            if index > 0:
                year = self.df_m2["year"][index]
                first_value = self.df_m2["average_m2"][0]
                
                start_value = self.df_m2["average_m2"][index-1]
                end_value = self.df_m2["average_m2"][index]
                resultado = (end_value - first_value) / first_value * 100
                
                print(f'{year}: {resultado}')

    def grafico_de_linha(self):
        figure = plt.figure(
            figsize=(10, 8)
        )

        plt.plot(
            self.df_gold_price['Year'],
            self.df_gold_price['Average_Price'],
        )

        plt.savefig('../img/grafico_de_linha.png')
 

In [75]:
if __name__ == "__main__":
    gold_analysis = GoldAnalysis()
    gold_analysis.increase_in_new_dollar_creation_per_year_in_percentage()


1960: 3.7447437208773775
1961: 10.751221729742005
1962: 19.365837026934862
1963: 29.42663939083987
1964: 39.5556313217411
1965: 50.852369587453104
1966: 60.71428571428571
1967: 71.7155358563473
1968: 85.91885441527448
1969: 97.31503579952268
1970: 105.06591658142972
1971: 129.93521991135356
1972: 158.49244232299122
1973: 183.58620297761107
1974: 200.22445732469603
1975: 228.50323900443232
1976: 270.4511876349585
1977: 316.3626548471417
1978: 350.81259234003863
1979: 386.0779634049324
1980: 425.12217297420165
1981: 472.5508580520514
1982: 524.2499147630414
1983: 600.4659620411411
1984: 656.5490396635981
1985: 723.9004432321854
1986: 790.8199795431299
1987: 848.6049551085349
1988: 899.357881577452
1989: 941.2632117286055
1990: 998.6134788044095
1991: 1039.4817592908284
1992: 1060.7796340493237
1993: 1072.7781566087053
1994: 1087.6406409819297
1995: 1112.2684395954086
1996: 1171.3774292533242
1997: 1238.4674394817591
1998: 1334.182861688828
1999: 1440.149448801
2000: 1533.202636663257
200